In [1]:
!pip install -q transformers datasets accelerate bitsandbytes wandb peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

import huggingface_hub
huggingface_hub.login()


In [3]:
import os
import time
import wandb
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset, Dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from peft import LoraConfig, get_peft_model, LoraConfig, TaskType

# wandb 초기화
wandb.init(project="Hanghae99_week8_advanced", name="gemma-2-2b-it_lightweight_experiment")

# 하이퍼파라미터/설정값
MODEL_NAME = "google/gemma-2-2b-it"
lora_r = 8

data_path = "corpus.json"

with open(data_path, "r", encoding="utf-8") as f:
    corpus = json.load(f)

dataset = Dataset.from_dict({
    "question": [item["question"] for item in corpus],
    "answer": [item["answer"] for item in corpus]
})

train_split = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_dataset = train_split['train']
eval_dataset = train_split['test']

# ===== 모델과 토크나이저 로드 (양자화 적용) =====
print(f"Loading model {MODEL_NAME} with 8bit quantization...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# 모델 파라미터 수 로깅
n_params = sum(p.numel() for p in model.parameters())
wandb.log({f"{MODEL_NAME}_num_params": n_params})
print(f"{MODEL_NAME} parameters: {n_params}")

# ===== 평가용 함수 정의 =====
def evaluate_inference_speed(model, tokenizer, prompt, num_runs=3):
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    times = []
    for i in range(num_runs):
        start = time.time()
        _ = generator(prompt, max_length=100)
        elapsed = time.time() - start
        times.append(elapsed)
        print(f"Run {i+1}: {elapsed:.3f} sec")
    avg_time = sum(times) / len(times)
    return avg_time

# ===== 사전 평가 (Fine-tuning 전 평가) =====
# 정량 평가
instruction = "How does the VIX (Volatility Index) influence trading decisions in the stock market"

print("\nPerforming quantitative evaluation (pre fine-tuning)...")

# 정량적 평가: 추론 속도 측정
avg_time = evaluate_inference_speed(model, tokenizer, instruction)
wandb.log({f"{MODEL_NAME}_avg_inference_time_pre": avg_time})
print(f"Avg Inference Time for {MODEL_NAME} (pre): {avg_time:.3f} sec")

# 정성적 평가: 동일 instruction에 대한 응답 생성
device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
print("\nPerforming qualitative evaluation (pre fine-tuning)...")
output = generator(instruction, max_length=250)
response_text = output[0]['generated_text']
wandb.log({f"{MODEL_NAME}_response_pre": response_text})
print(f"Response from {MODEL_NAME} (pre):\n{response_text}")
print("-" * 60)

# ===== Fine-tuning을 위한 준비 =====
def formatting_prompts_func(data):
    prompt_list = []
    for i in range(len(data['question'])):
        prompt_list.append(f"""<bos><start_of_turn>user
{data['question'][i]}<end_of_turn>
<start_of_turn>model
{data['answer'][i]}<end_of_turn><eos>""")
    return prompt_list


response_template = "<start_of_turn>model"

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# ===== LoRA 적용 =====

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=lora_r,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
)


model = get_peft_model(model, lora_config)

# ===== Fine-tuning 설정 및 실행 =====
training_config = SFTConfig(
    output_dir=f"./gemma-2-2b-it_output",
    save_total_limit=1,
    logging_steps=100,
    eval_steps=100,
    max_steps=1000,
    fp16=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=100,
    save_strategy="steps",
    logging_strategy="steps",
    evaluation_strategy="steps",
)

trainer = SFTTrainer(
    model=model,
    args=training_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

print(f"\nStarting fine-tuning for {MODEL_NAME} ...")
train_start_time = time.time()
trainer.train()
train_end_time = time.time()
total_training_time = (train_end_time - train_start_time) / 60
wandb.log({"train/total_training_time_min": total_training_time})
print(f"Fine-tuning for {MODEL_NAME} completed.")

# ===== 9. Fine-tuning 후 재평가 =====
print("\nPerforming quantitative evaluation (post fine-tuning)...")
avg_time_post = evaluate_inference_speed(model, tokenizer, instruction)
wandb.log({f"{MODEL_NAME}_avg_inference_time_post": avg_time_post})
print(f"Avg Inference Time for {MODEL_NAME} (post): {avg_time_post:.3f} sec")

print("\nPerforming qualitative evaluation (post fine-tuning)...")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
output_post = generator(instruction, max_length=250)
response_text_post = output_post[0]['generated_text']
wandb.log({f"{MODEL_NAME}_response_post": response_text_post})
print(f"Response from {MODEL_NAME} (post):\n{response_text_post}")
print("-" * 60)

# ===== 10. 마무리 =====
wandb.finish()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: code-sugartoast (code-sugartoast-sugartoast) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Loading model google/gemma-2-2b-it with 8bit quantization...


...

The 'batch_size' argument of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausa

Fine-tuning for google/gemma-2-2b-it completed.

Performing quantitative evaluation (post fine-tuning)...
Run 1: 2.743 sec
Run 2: 2.614 sec


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

Run 3: 2.728 sec
Avg Inference Time for google/gemma-2-2b-it (post): 2.695 sec

Performing qualitative evaluation (post fine-tuning)...
Response from google/gemma-2-2b-it (post):
How does the VIX (Volatility Index) influence trading decisions in the stock market?
The VIX measures market expectations of volatility, and traders use it to gauge market sentiment and anticipate potential price movements.
------------------------------------------------------------


eval/loss,█▆▄▃▃▂▂▁▁▁
eval/mean_token_accuracy,▁▃▅▅▆▇▇███
eval/runtime,▂▃▁▅█▇▇▇▇█
eval/samples_per_second,▇▆█▄▁▂▂▂▂▁
eval/steps_per_second,▇▆█▄▁▂▂▂▂▁
google/gemma-2-2b-it_avg_inference_time_post,▁
google/gemma-2-2b-it_avg_inference_time_pre,▁
google/gemma-2-2b-it_num_params,▁
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██████
train/grad_norm,▁▅▂▄█▆▃▇▅▅
